In [1]:
#from bs4 import BeautifulSoup
import re
import xml.etree.ElementTree as ET
from attribute import Attribute as att
from copy import deepcopy

In [2]:
my_parser = ET.XMLParser(encoding='utf-8')
tree = ET.parse('dicom-2020a.dicomextract.txt', parser=my_parser)
root = tree.getroot()

In [3]:
c=[]
c = root.findall(".//ItemEntry[@Type='1C']") + root.findall(".//ItemEntry[@Type='2C']")

In [4]:
#Dictionary with conditional attributes:
attribute_dict = []
for a in c:
    description = a.attrib['Description']
    tag = a.attrib['Tag']
    type = a.attrib['Type']
    attribute_dicom = att(description,tag, type)
    attribute_dict.append(attribute_dicom)
attribute_original = deepcopy(attribute_dict)

In [5]:
# Remove Notes, See Section, May be present
print(attribute_original[60].description)
for a in attribute_dict:
    a.clean_note()
    a.clean_section()
    a.clean_may_be_present()
print(attribute_dict[60].description)

The floating point representation of Numeric Value (0040,A30A). The same number of values as Numeric Value (0040,A30A) shall be present. Required if Numeric Value (0040,A30A) has insufficient precision to represent the value as a string. May be present otherwise.
The floating point representation of Numeric Value (0040,A30A). The same number of values as Numeric Value (0040,A30A) shall be present. Required if Numeric Value (0040,A30A) has insufficient precision to represent the value as a string. 


In [30]:
for a in attribute_dict:
    a.set_only_rule_def()
#print(attribute_original[2].description)
for i in attribute_dict:
    print(i.description)

Required if an extended or replacement character set is used in one of the keys
Required if an extended or replacement character set is used in one of the keys
Required only if (0004,1511) is absent (see 
Required if an extended or replacement character set is used in one of the keys
Required if an extended or replacement character set is used in one of the keys.
Required if an extended or replacement character set is used in one of the keys.
Required if an extended or replacement character set is used in one of the keys.
Required if an extended or replacement character set is used in one of the keys
Required if an extended or replacement character set is used in one of the keys
Required if an extended or replacement character set is used in one of the keys.
Required if the IOD of the Presentation SOP Instance referenced by this directory record includes the Presentation State Relationship Module (PS3.3:sect_C.11.11).
Required if the SOP Instance referenced by this directory record inc

In [42]:
description = 'Required if object extraction is based on a Frame Level Retrieve using the Simple Frame List (0008,1161) Attribute. See PS3.4 "Instance and Frame Level Retrieve SOP Classes'
req = re.search('(Required.*)((\\.)|($))', description)
print(req.group())

Required if object extraction is based on a Frame Level Retrieve using the Simple Frame List (0008,1161) Attribute. See PS3.4 "Instance and Frame Level Retrieve SOP Classes


In [7]:
required = []
for a in range(len(attribute_dict)):
    if 'Shall' in attribute_dict[a].description:
        required.append(a)
        print(a)
print(len(required))

41
42
43
44
45
485
486
521
626
627
648
652
653
654
655
656
749
759
897
1014
1092
1517
1598
23


In [8]:
b = attribute_dict[1718]
print(b.tag, b.description, b.type)

(0054,0032) Required if the Frame Increment Pointer (0028,0009) contains the Tag for Phase Vector (0054,0030). Zero or more Items shall be included in this Sequence. The number of Items shall be equal to Number of Phases (0054,0031). The first Item corresponds to frames with value of 1 in Phase Vector (0054,0030), the second Item with value 2, etc. 2C


In [9]:
#define method to look for See Section 10.1020.301230
#define method to look for May be present otherwise
#define method for one of more items shall be included